### LD Matrix Numba vs Cuda

Compare 1KG LD matrix calc times on CPU and GPU

In [1]:
from lib import api
import xarray as xr
import numpy as np
%run ../nb/paths.py

In [2]:
path = PLINK_1KG_PATH_02
path

PosixPath('/lab/data/gwas/tutorial/2_PS_GWAS/1kG_MDS5')

In [3]:
%%time
ds = api.read_plink(path, chunks='auto', fam_sep=' ', bim_sep='\t', lock=False)
ds

CPU times: user 36.6 s, sys: 4.17 s, total: 40.8 s
Wall time: 25.7 s


<xarray.Dataset>
Dimensions:     (sample: 629, variant: 5808310)
Coordinates:
  * variant     (variant) int64 0 1 2 3 4 ... 5808306 5808307 5808308 5808309
  * sample      (sample) int64 0 1 2 3 4 5 6 7 ... 622 623 624 625 626 627 628
Data variables:
    data        (variant, sample) int8 dask.array<chunksize=(213382, 629), meta=np.ndarray>
    is_masked   (variant, sample) bool dask.array<chunksize=(213382, 629), meta=np.ndarray>
    sample_id   (sample) int64 dask.array<chunksize=(629,), meta=np.ndarray>
    fam_id      (sample) object dask.array<chunksize=(629,), meta=np.ndarray>
    pat_id      (sample) int64 dask.array<chunksize=(629,), meta=np.ndarray>
    mat_id      (sample) int64 dask.array<chunksize=(629,), meta=np.ndarray>
    is_female   (sample) int64 dask.array<chunksize=(629,), meta=np.ndarray>
    phenotype   (sample) int64 dask.array<chunksize=(629,), meta=np.ndarray>
    contig      (variant) int64 dask.array<chunksize=(1948852,), meta=np.ndarray>
    variant_id  (variant) object dask.array<chunksize=(1948852,), meta=np.ndarray>
    cm_pos      (variant) int64 dask.array<chunksize=(1948852,), meta=np.ndarray>
    pos         (variant) int64 dask.array<chunksize=(1948852,), meta=np.ndarray>
    a1          (variant) object dask.array<chunksize=(1948852,), meta=np.ndarray>
    a2          (variant) object dask.array<chunksize=(1948852,), meta=np.ndarray>
Attributes:
    type:     GenotypeCountDataset

In [4]:
# Set intervals for LD calculation (standard parameterization is 1000kbp window)
intervals = api.axis_intervals(ds, window=1_000_000, unit='physical', target_chunk_size=100_000, backend='numba')

In [6]:
# Average chunk size
(int(intervals[1].pipe(lambda a: a.sel(var='max_stop') - a.sel(var='min_start')).mean().values), ds.dims['sample'])

(86922, 629)

In [9]:
%%time
ldm = api.ld_matrix(ds, intervals=intervals, threshold=None, backend='dask/numba')
ldm.count().compute(scheduler='threads', num_workers=8, threads_per_worker=1)

CPU times: user 4h 8min 36s, sys: 1min 40s, total: 4h 10min 17s
Wall time: 33min 19s


i        13414996395
j        13414996395
value    13414996395
dtype: int64

In [19]:
%%time
ldm = api.ld_matrix(ds, intervals=intervals, threshold=None, backend='dask/cuda')
ldm.count().compute(scheduler='single-threaded')

CPU times: user 23min 35s, sys: 1min 34s, total: 25min 10s
Wall time: 25min 7s


i        13414996395
j        13414996395
value    13414996395
dtype: int64

In [7]:
%%time
ldm = api.ld_matrix(ds, intervals=intervals, threshold=0.2, backend='dask/cuda')
ldm.count().compute(scheduler='single-threaded')

CPU times: user 22min 10s, sys: 31.8 s, total: 22min 41s
Wall time: 22min 39s


i        371826627
j        371826627
value    371826627
dtype: int64